In [146]:
import sqlite3
import pandas as pd
import numpy as np
from difflib import 

In [2]:
conn = sqlite3.connect('zotero.sqlite')

purpose: get title, main author, number of authors, year for each entry in a collection

### get a collection id for a known collection name



In [4]:
coll_id_query = """select collectionid
from collections 
where collectionname= ?"""

In [13]:
get_collection_id = lambda cid: list(conn.execute(coll_id_query, (cid,)))[0][0]


In [14]:
get_collection_id("choi2004optimal")

24

### get id of title and date fields

In [59]:
list(conn.execute("select * from fields where fieldname like '%pub%'"))

[(23, 'publisher', None),
 (37, 'publicationTitle', None),
 (99, 'publicLawNumber', None)]

In [60]:
title_date_id_query = """
select 
    sum(iif(fieldname="publicationTitle",fieldid,0)) publication_id,
    sum(iif(fieldname="title",fieldid,0)) title_id,
    sum(iif(fieldname="date",fieldid,0)) date_id
    from fields"""

In [61]:
get_title_and_date_id = lambda: list(conn.execute(title_date_id_query))[0]

In [62]:
publication_id,title_id, date_id = get_title_and_date_id()

In [63]:
publication_id,title_id, date_id

(37, 1, 6)

### get all titles and dates in a collection


In [64]:
title_and_date = """
    select ci.itemId,
    group_concat(iif(fieldid={},value,null)) title,
    group_concat(iif(fieldid={},value,null)) publication,
    group_concat(iif(fieldid={},value,null)) date
    from itemData it
    inner join itemDataValues idv on it.valueId=idv.valueId
    inner join collectionItems ci on ci.itemId = it.itemId
    where ci.collectionId = ?
    group by ci.itemId"""

In [65]:
get_item_tilte_and_date_in_collection = lambda collid: \
    list(conn.execute(title_and_date.format(title_id, publication_id, date_id),(collid,)))

In [66]:
get_item_tilte_and_date_in_collection(24)

[(294,
  'Pricing and product design: Intermediary strategies in an electronic market',
  'Working Paper',
  '2001-00-00 2001'),
 (297,
  'Frictionless commerce? A comparison of Internet and conventional retailers',
  'Management Science',
  '2000-00-00 2000'),
 (300, None, 'E-Business and E-Commerce Management', '2002-00-00 2002'),
 (303,
  'B2B emarketplace announcements and shareholder wealth',
  'Economic and Financial Review',
  '2001-00-00 2001'),
 (306,
  'Designing robust, open electronic marketplaces of contract net agents',
  'Working Paper',
  '2002-00-00 2002'),
 (309,
  'Efficient Supply Contracts for Fashion Goods with Forecast Updating and Two Production Modes',
  'Management Science',
  '2000-00-00 2000'),
 (312,
  'Note. The role of returns policies in pricing and inventory decisions for catalogue goods',
  'Management Science',
  '1998-00-00 1998'),
 (315,
  'Making business sense of the Internet.',
  'Harvard business review',
  '1998-00-00 1998'),
 (318,
  'E-commer

In [97]:
creator_count_and_first = lambda how_many_params:"""
select itemId, num_creators, lastname
from (
    select itemId,
    count(creatorid) num_creators,
    sum(iif(orderIndex==0,creatorid,0)) first_creator_id
    from ItemCreators
    group by itemId) a
inner join creators on creators.creatorid=a.first_creator_id
where itemid in ({})""".format(('?,'*how_many_params)[:-1])

In [96]:
list(conn.execute(creator_count_and_first(35),item_ids))

[(294, 3, 194, 'Bhargava'),
 (297, 2, 197, 'Brynjolfsson'),
 (300, 1, 199, 'Chaffey'),
 (303, 2, 200, 'Chen'),
 (306, 2, 202, 'Dellarocas'),
 (309, 1, 204, 'Donohue'),
 (312, 2, 205, 'Emmons'),
 (315, 1, 207, 'Ghosh'),
 (318, 4, 208, 'Gunasekaran'),
 (321, 1, 212, 'Haller'),
 (324, 5, 213, 'Kafka'),
 (327, 1, 218, 'Kandel'),
 (330, 2, 219, 'Kannan'),
 (333, 2, 221, 'Kaplan'),
 (336, 2, 223, 'Kauffman'),
 (339, 2, 225, 'Keskinocak'),
 (342, 2, 227, 'Lau'),
 (345, 3, 228, 'Lau'),
 (348, 2, 102, 'Lau'),
 (351, 2, 230, 'Laudon'),
 (354, 2, 232, 'Law'),
 (357, 2, 234, 'Lucking-Reiley'),
 (360, 1, 236, 'Markowitz'),
 (363, 3, 237, 'Ngai'),
 (366, 2, 96, 'Netessine'),
 (369, 1, 241, 'Pasternack'),
 (372, 2, 242, 'Petruzzi'),
 (375, 2, 244, 'Sarkis'),
 (378, 1, 246, 'Sodhi'),
 (381, 1, 247, 'Spengler'),
 (384, 1, 248, 'Stefansson'),
 (387, 4, 249, 'Turban'),
 (390, 1, 253, 'Ward'),
 (393, 2, 254, 'Webster'),
 (396, 1, 256, 'Yau')]

In [80]:
item_ids = [a[0] for a in get_item_tilte_and_date_in_collection(24)]

In [98]:
get_authors_for_item_ids= lambda iids: list(conn.execute(creator_count_and_first(len(iids)),iids))

In [91]:
len(item_ids)

35

In [139]:
def get_publication_nicknames_for_collection(collection_id=None, name=None):
    # run the two queries: the one with the id, title and date
    # and the one with the id, number of authors, and name of first author
    # join using itemId and replace the title with publication if title is none
    # return a list of itemid, nickname where nickname is author+year+first word of title

    # find collection_id if not provided
    if collection_id is None:
        if name is None:
            raise ValueError("Must supply either collection_id or collection_name")
        
        coll_id_query = """ select collectionid
                            from collections 
                            where collectionname= ?"""

        get_collection_id = lambda cid: list(conn.execute(coll_id_query, (cid,)))[0][0]
        collection_id = get_collection_id(name)
    
    # get the fieldId for the required fields
    title_date_id_query = """
    select 
    sum(iif(fieldname="publicationTitle",fieldid,0)) publication_id,
    sum(iif(fieldname="title",fieldid,0)) title_id,
    sum(iif(fieldname="date",fieldid,0)) date_id
    from fields
    """
    get_title_and_date_id = lambda: list(conn.execute(title_date_id_query))[0]
    publication_id,title_id, date_id = get_title_and_date_id()
    
    
    # get the items in the collection
    title_and_date = """
    select ci.itemId,
    group_concat(iif(fieldid={},value,null)) title,
    group_concat(iif(fieldid={},value,null)) publication,
    group_concat(iif(fieldid={},value,null)) date
    from itemData it
    inner join itemDataValues idv on it.valueId=idv.valueId
    inner join collectionItems ci on ci.itemId = it.itemId
    left join deletedItems di on it.itemId=di.itemId
    where ci.collectionId = ?
    and di.itemId is null
    group by ci.itemId"""

    get_item_title_and_date_in_collection = lambda collid: \
        list(conn.execute(title_and_date.format(title_id, publication_id, date_id),(collid,)))

    items = get_item_title_and_date_in_collection(collection_id)
    
    item_ids = [a[0] for a in items]
    
    
    
    creator_count_and_first = lambda how_many_params:"""
        select itemId, num_creators, lastname
        from (
            select itemId,
            count(creatorid) num_creators,
            sum(iif(orderIndex==0,creatorid,0)) first_creator_id
            from ItemCreators
            group by itemId) a
        inner join creators on creators.creatorid=a.first_creator_id
        where itemid in ({})""".format(('?,'*how_many_params)[:-1])
    
    get_authors_for_item_ids= lambda iids: list(conn.execute(creator_count_and_first(len(iids)),iids))
    
    author_dict = {
        item_id: {
            "num_authors": num_authors, 
            "first_author": first_author
        } for item_id, num_authors, first_author in get_authors_for_item_ids(item_ids)}
    
    items_in_collection = { 
        item_id: {
            "title": title or publication,
            "title_words": (title or publication).lower().split(' ')[:4],
            "year": date[:4],
            "num_authors": author_dict[item_id]['num_authors'],
            "first_author": author_dict[item_id]['first_author'],
        }
    
    for item_id, title, publication,date in items
    }
    
    nonwords = {'the','a','on','do','does','in','note.'}
    
    for item_id, item_data in items_in_collection.items():
        item_data['nickname']="{}{}{}".format(
            item_data['first_author'].lower(), 
            item_data['year'], 
            [''.join(c for c in word if c.isalnum()) for word in item_data['title_words'] if word not in nonwords][0]
        ).replace(' ','_')
    
    return items_in_collection

In [140]:
pd.DataFrame(get_publication_nicknames_for_collection(name='choi2004optimal')).T

,title,title_words,year,num_authors,first_author,nickname
294,Pricing and product design: Intermediary strat...,"[pricing, and, product, design:]",2001,3,Bhargava,bhargava2001pricing
297,Frictionless commerce? A comparison of Interne...,"[frictionless, commerce?, a, comparison]",2000,2,Brynjolfsson,brynjolfsson2000frictionless
300,E-Business and E-Commerce Management,"[e-business, and, e-commerce, management]",2002,1,Chaffey,chaffey2002ebusiness
303,B2B emarketplace announcements and shareholder...,"[b2b, emarketplace, announcements, and]",2001,2,Chen,chen2001b2b
306,"Designing robust, open electronic marketplaces...","[designing, robust,, open, electronic]",2002,2,Dellarocas,dellarocas2002designing
309,Efficient Supply Contracts for Fashion Goods w...,"[efficient, supply, contracts, for]",2000,1,Donohue,donohue2000efficient
312,Note. The role of returns policies in pricing ...,"[note., the, role, of]",1998,2,Emmons,emmons1998role
315,Making business sense of the Internet.,"[making, business, sense, of]",1998,1,Ghosh,ghosh1998making
318,E-commerce and its impact on operations manage...,"[e-commerce, and, its, impact]",2002,4,Gunasekaran,gunasekaran2002ecommerce
321,The impact of electronic markets on B2B relati...,"[the, impact, of, electronic]",2002,1,Haller,haller2002impact


In [142]:
base_items = pd.DataFrame(get_publication_nicknames_for_collection(name='T_and_T')).T
base_items

,title,title_words,year,num_authors,first_author,nickname
1,Revenue-sharing between developers of virtual ...,"[revenue-sharing, between, developers, of]",2020,3,Avinadav,avinadav2020revenuesharing
243,Optimal returns policy for supply chain with e...,"[optimal, returns, policy, for]",2004,3,Choi,choi2004optimal
245,Green supply chain contracts with eco-labels i...,"[green, supply, chain, contracts]",2020,3,Guo,guo2020green
249,Blockchain and smart contracts in supply chain...,"[blockchain, and, smart, contracts]",2020,1,De Giovanni,degiovanni2020blockchain
250,Competition in the multi-sided platform market...,"[competition, in, the, multi-sided]",2017,3,de Matta,dematta2017competition
252,Reducing channel costs by investing in smart s...,"[reducing, channel, costs, by]",2020,1,Li,li2020reducing
253,The impacts of market size and data-driven mar...,"[the, impacts, of, market]",2020,4,Liu,liu2020impacts
254,Selling secondhand products through an online ...,"[selling, secondhand, products, through]",2020,3,Shen,shen2020selling
256,Supply chain finance: A systematic literature ...,"[supply, chain, finance:, a]",2018,6,Xu,xu2018supply
257,Strategic introduction of the marketplace chan...,"[strategic, introduction, of, the]",2018,3,Yan,yan2018strategic


In [144]:
base_items.assign('associated_collection'="""
Avinadav2020Revenue-sharing
choi2004optimal
"""
    
)

SyntaxError: invalid syntax (<ipython-input-144-f99b82a91778>, line 3)

In [155]:
collection_names=[a[0] for a in conn.execute("select collectionname from collections")]
collection_names

['T_and_T',
 'Avinadav2020Revenue-sharing',
 'choi2004optimal',
 'de_giovanni2020blockchain',
 'deMatta2017Competition',
 'grieger2003electronic',
 'guo2020green',
 'jiang2017p2p',
 'Li2019e-tailers',
 'Li2020reducing',
 'Liu2020impacts',
 'mantin2014strategic',
 'ryan2012competition',
 'Shen2020selling',
 'tian2018marketplace',
 'zheng2020manufacturer',
 'zhang2020agency',
 'zenyo2020strategic',
 'yan2019strategic',
 'yan2018strategic',
 'xu2018supply',
 'Xiu2020coordination']

In [150]:
base_items.nickname.map(lambda v:max((SequenceMatcher(None,v,name).ratio(),name) for name in collection_names)) 

1      (0.0, (zheng2020manufacturer,))
243    (0.0, (zheng2020manufacturer,))
245    (0.0, (zheng2020manufacturer,))
249    (0.0, (zheng2020manufacturer,))
250    (0.0, (zheng2020manufacturer,))
252    (0.0, (zheng2020manufacturer,))
253    (0.0, (zheng2020manufacturer,))
254    (0.0, (zheng2020manufacturer,))
256    (0.0, (zheng2020manufacturer,))
257    (0.0, (zheng2020manufacturer,))
258    (0.0, (zheng2020manufacturer,))
259    (0.0, (zheng2020manufacturer,))
260    (0.0, (zheng2020manufacturer,))
272    (0.0, (zheng2020manufacturer,))
275    (0.0, (zheng2020manufacturer,))
277    (0.0, (zheng2020manufacturer,))
279    (0.0, (zheng2020manufacturer,))
281    (0.0, (zheng2020manufacturer,))
285    (0.0, (zheng2020manufacturer,))
288    (0.0, (zheng2020manufacturer,))
290    (0.0, (zheng2020manufacturer,))
Name: nickname, dtype: object

In [158]:
base_items.nickname.map(lambda v:difflib.get_close_matches(v,collection_names)) 

1                          [Avinadav2020Revenue-sharing]
243                                    [choi2004optimal]
245                                       [guo2020green]
249                          [de_giovanni2020blockchain]
250        [deMatta2017Competition, ryan2012competition]
252                                     [Li2020reducing]
253                                     [Liu2020impacts]
254                                    [Shen2020selling]
256                                       [xu2018supply]
257    [yan2018strategic, yan2019strategic, mantin201...
258    [zenyo2020strategic, yan2019strategic, yan2018...
259             [zhang2020agency, zheng2020manufacturer]
260                              [zheng2020manufacturer]
272                              [grieger2003electronic]
275                                       [jiang2017p2p]
277                                    [Li2019e-tailers]
279    [mantin2014strategic, yan2019strategic, yan201...
281        [ryan2012competition

In [161]:
base_items['ref_collection_name']=base_items.nickname.map(lambda v:difflib.get_close_matches(v,collection_names)[0]) 
base_items

,title,title_words,year,num_authors,first_author,nickname,ref_collection_name
1,Revenue-sharing between developers of virtual ...,"[revenue-sharing, between, developers, of]",2020,3,Avinadav,avinadav2020revenuesharing,Avinadav2020Revenue-sharing
243,Optimal returns policy for supply chain with e...,"[optimal, returns, policy, for]",2004,3,Choi,choi2004optimal,choi2004optimal
245,Green supply chain contracts with eco-labels i...,"[green, supply, chain, contracts]",2020,3,Guo,guo2020green,guo2020green
249,Blockchain and smart contracts in supply chain...,"[blockchain, and, smart, contracts]",2020,1,De Giovanni,degiovanni2020blockchain,de_giovanni2020blockchain
250,Competition in the multi-sided platform market...,"[competition, in, the, multi-sided]",2017,3,de Matta,dematta2017competition,deMatta2017Competition
252,Reducing channel costs by investing in smart s...,"[reducing, channel, costs, by]",2020,1,Li,li2020reducing,Li2020reducing
253,The impacts of market size and data-driven mar...,"[the, impacts, of, market]",2020,4,Liu,liu2020impacts,Liu2020impacts
254,Selling secondhand products through an online ...,"[selling, secondhand, products, through]",2020,3,Shen,shen2020selling,Shen2020selling
256,Supply chain finance: A systematic literature ...,"[supply, chain, finance:, a]",2018,6,Xu,xu2018supply,xu2018supply
257,Strategic introduction of the marketplace chan...,"[strategic, introduction, of, the]",2018,3,Yan,yan2018strategic,yan2018strategic


In [162]:
base_items['ref_collection_id']=base_items.ref_collection_name.map(lambda v: get_collection_id(v))
base_items

,title,title_words,year,num_authors,first_author,nickname,ref_collection_name,ref_collection_id
1,Revenue-sharing between developers of virtual ...,"[revenue-sharing, between, developers, of]",2020,3,Avinadav,avinadav2020revenuesharing,Avinadav2020Revenue-sharing,2
243,Optimal returns policy for supply chain with e...,"[optimal, returns, policy, for]",2004,3,Choi,choi2004optimal,choi2004optimal,24
245,Green supply chain contracts with eco-labels i...,"[green, supply, chain, contracts]",2020,3,Guo,guo2020green,guo2020green,28
249,Blockchain and smart contracts in supply chain...,"[blockchain, and, smart, contracts]",2020,1,De Giovanni,degiovanni2020blockchain,de_giovanni2020blockchain,25
250,Competition in the multi-sided platform market...,"[competition, in, the, multi-sided]",2017,3,de Matta,dematta2017competition,deMatta2017Competition,26
252,Reducing channel costs by investing in smart s...,"[reducing, channel, costs, by]",2020,1,Li,li2020reducing,Li2020reducing,31
253,The impacts of market size and data-driven mar...,"[the, impacts, of, market]",2020,4,Liu,liu2020impacts,Liu2020impacts,32
254,Selling secondhand products through an online ...,"[selling, secondhand, products, through]",2020,3,Shen,shen2020selling,Shen2020selling,35
256,Supply chain finance: A systematic literature ...,"[supply, chain, finance:, a]",2018,6,Xu,xu2018supply,xu2018supply,42
257,Strategic introduction of the marketplace chan...,"[strategic, introduction, of, the]",2018,3,Yan,yan2018strategic,yan2018strategic,41


In [183]:
cross_collection_query = """
select c.itemId,count(1) cnt
from collectionItems c
left join deletedItems di on c.itemId=di.itemId
where collectionId in (?,?)
and di.itemId is null
group by c.itemId
having cnt>1
"""

get_joint_reference_count = lambda cid1,cid2: len(list(conn.execute(cross_collection_query,(cid1,cid2))))

In [192]:
get_joint_reference_count(2,2)

0

In [194]:
count_joint_reference = \
    lambda u,v: len(set(get_item_tilte_and_date_in_collection(u)).intersection(get_item_tilte_and_date_in_collection(v))) 
base_items['joint_reference_count'] = \
    base_items.ref_collection_id.map(
    lambda v: [ count_joint_reference(u,v) for u in base_items.ref_collection_id])
base_items

,title,title_words,year,num_authors,first_author,nickname,ref_collection_name,ref_collection_id,joint_reference_count,reference_count
1,Revenue-sharing between developers of virtual ...,"[revenue-sharing, between, developers, of]",2020,3,Avinadav,avinadav2020revenuesharing,Avinadav2020Revenue-sharing,2,"[80, 0, 1, 0, 2, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,...",80
243,Optimal returns policy for supply chain with e...,"[optimal, returns, policy, for]",2004,3,Choi,choi2004optimal,choi2004optimal,24,"[0, 35, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,...",35
245,Green supply chain contracts with eco-labels i...,"[green, supply, chain, contracts]",2020,3,Guo,guo2020green,guo2020green,28,"[1, 0, 58, 0, 0, 2, 2, 1, 0, 1, 4, 1, 1, 0, 0,...",58
249,Blockchain and smart contracts in supply chain...,"[blockchain, and, smart, contracts]",2020,1,De Giovanni,degiovanni2020blockchain,de_giovanni2020blockchain,25,"[0, 0, 0, 54, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0,...",54
250,Competition in the multi-sided platform market...,"[competition, in, the, multi-sided]",2017,3,de Matta,dematta2017competition,deMatta2017Competition,26,"[2, 0, 0, 0, 35, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0,...",35
252,Reducing channel costs by investing in smart s...,"[reducing, channel, costs, by]",2020,1,Li,li2020reducing,Li2020reducing,31,"[1, 0, 2, 1, 1, 57, 9, 9, 0, 4, 4, 1, 3, 0, 0,...",57
253,The impacts of market size and data-driven mar...,"[the, impacts, of, market]",2020,4,Liu,liu2020impacts,Liu2020impacts,32,"[1, 0, 2, 0, 1, 9, 53, 3, 0, 5, 7, 1, 3, 0, 1,...",53
254,Selling secondhand products through an online ...,"[selling, secondhand, products, through]",2020,3,Shen,shen2020selling,Shen2020selling,35,"[0, 0, 1, 2, 0, 9, 3, 70, 0, 1, 2, 0, 2, 0, 1,...",70
256,Supply chain finance: A systematic literature ...,"[supply, chain, finance:, a]",2018,6,Xu,xu2018supply,xu2018supply,42,"[0, 0, 0, 0, 0, 0, 0, 0, 85, 0, 0, 0, 5, 0, 0,...",85
257,Strategic introduction of the marketplace chan...,"[strategic, introduction, of, the]",2018,3,Yan,yan2018strategic,yan2018strategic,41,"[1, 0, 1, 0, 0, 4, 5, 1, 0, 71, 5, 3, 6, 0, 1,...",71


In [191]:
base_items['reference_count'] = [len(get_item_tilte_and_date_in_collection(i)) for i in base_items.ref_collection_id]
base_items

,title,title_words,year,num_authors,first_author,nickname,ref_collection_name,ref_collection_id,joint_reference_count,reference_count
1,Revenue-sharing between developers of virtual ...,"[revenue-sharing, between, developers, of]",2020,3,Avinadav,avinadav2020revenuesharing,Avinadav2020Revenue-sharing,2,"[0, 0, 1, 0, 2, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, ...",80
243,Optimal returns policy for supply chain with e...,"[optimal, returns, policy, for]",2004,3,Choi,choi2004optimal,choi2004optimal,24,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",35
245,Green supply chain contracts with eco-labels i...,"[green, supply, chain, contracts]",2020,3,Guo,guo2020green,guo2020green,28,"[1, 0, 0, 0, 0, 2, 2, 1, 0, 1, 4, 1, 1, 0, 0, ...",58
249,Blockchain and smart contracts in supply chain...,"[blockchain, and, smart, contracts]",2020,1,De Giovanni,degiovanni2020blockchain,de_giovanni2020blockchain,25,"[0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, ...",54
250,Competition in the multi-sided platform market...,"[competition, in, the, multi-sided]",2017,3,de Matta,dematta2017competition,deMatta2017Competition,26,"[2, 0, 0, 0, 0, 1, 1, 0, 0, 0, 3, 0, 0, 0, 0, ...",35
252,Reducing channel costs by investing in smart s...,"[reducing, channel, costs, by]",2020,1,Li,li2020reducing,Li2020reducing,31,"[1, 0, 2, 1, 1, 0, 9, 9, 0, 4, 4, 1, 3, 0, 0, ...",57
253,The impacts of market size and data-driven mar...,"[the, impacts, of, market]",2020,4,Liu,liu2020impacts,Liu2020impacts,32,"[1, 0, 2, 0, 1, 9, 0, 3, 0, 5, 7, 1, 3, 0, 1, ...",53
254,Selling secondhand products through an online ...,"[selling, secondhand, products, through]",2020,3,Shen,shen2020selling,Shen2020selling,35,"[0, 0, 1, 2, 0, 9, 3, 0, 0, 1, 2, 0, 2, 0, 1, ...",70
256,Supply chain finance: A systematic literature ...,"[supply, chain, finance:, a]",2018,6,Xu,xu2018supply,xu2018supply,42,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, ...",85
257,Strategic introduction of the marketplace chan...,"[strategic, introduction, of, the]",2018,3,Yan,yan2018strategic,yan2018strategic,41,"[1, 0, 1, 0, 0, 4, 5, 1, 0, 0, 5, 3, 6, 0, 1, ...",71


In [182]:
set(get_item_tilte_and_date_in_collection(43)).intersection(get_item_tilte_and_date_in_collection(40))

{(257,
  'Strategic introduction of the marketplace channel under spillovers from online to offline sales',
  'European Journal of Operational Research',
  '2018-05-00 05/2018'),
 (948,
  'Strategic Analysis of the Agency Model for Digital Goods',
  'Production and Operations Management',
  '2017-00-00 2017'),
 (1080,
  'Agency selling or reselling? Channel structures in electronic retailing',
  'Management Science',
  '2016-00-00 2016'),
 (1140, 'Marketplace or reseller?', 'Management Science', '2015-00-00 2015'),
 (2424,
  'Implementing coordination contracts in a manufacturer Stackelberg dual-channel supply chain',
  'Omega',
  '2012-00-00 2012')}

In [196]:
base_items.to_excel('cross_references_summary.xlsx',index_label='item_id')

In [197]:
pwd

'/home/siegman/Documents/Shahar/y2021/t_and_t'